In [1]:
import os, sys
sys.path.append(os.path.abspath('..'))
import pandas as pd
from src.services.mail_client import send_email
from src.parsers.base_parser import BaseParser
# from src.extract import INAVBskt
from src.parsers.inav_bskt import INAVBskt
from config.settings import DATA_DIR

In [2]:
file1 = BaseParser(os.path.join(DATA_DIR, 'PLF_Positions20260225.csv'))
df1 = file1.read_into_dataframe(header=0,index_col=False)

In [3]:
fil12 = INAVBskt(os.path.join(DATA_DIR, 'Harvest_INAVBSKT_ALL.20260218.CSV'))
df12 = fil12.extract()
# df12.to_csv(os.path.join(DATA_DIR, 'PLF_Positions20260225_cleaned.csv'), index=False)

In [4]:
fil12 = BaseParser(os.path.join(DATA_DIR, 'Harvest_INAVBSKT_ALL.20260218.CSV'))
df12 = fil12.list_to_blocks(fil12.read_into_list())
# ans = fil12.list_to_blocks()

In [5]:
news = pd.read_table(os.path.join(DATA_DIR, 'Harvest_INAVBSKT_ALL.20260218.CSV'),header=None)

In [ ]:
def _get_blocks(table:pd.DataFrame, start_marker:str="TRADE_DATE") -> list[pd.DataFrame]:
        """Split DataFrame into blocks based on rows that start with a marker."""
        try:
            starts = [i for i,v in enumerate(table.iloc[:,0]) if v.startswith(start_marker)]
            blocks = []
            for idx, start in enumerate(starts):
                if (idx + 1) < len(starts):
                    end = starts[idx + 1]
                else:
                    end = len(table)
                blocks.append(table[start:end].reset_index(drop=True))
            return blocks
        except Exception as e:
            raise ValueError(f"Error splitting table into blocks: {e}")
            return []


In [7]:
ans = _get_blocks(news)

In [9]:
print(ans[2])

                                                     0
108  TRADE_DATE,20260219,HR21UPR, ,Harvest Tech Ach...
109  CREATION_UNIT_SIZE,1000000,DOMICILE,"",ESTIMAT...
110  SETTLEMENT_CYCLE,T+1,ASSET_CLASS,"",ESTIMATED_...
111  NAV,20.51721,UNDISTRIBUTED_NET_INCOME_PER_SHAR...
112  NAV_PER_CREATION_UNIT,20517210,UNDISTRIBUTED_N...
113  NAV_LESS_UNDISTRIBUTED_NET_INCOME,20487210,ACT...
114  ETF_SHARES_OUTSTANDING,1775000,ACTUAL_INTEREST...
115  TOTAL_NET_ASSETS,36418056.4222,ACTUAL_TOTAL_CA...
116  CUSIP,TICKER,SEDOL,ISIN,DESCRIPTION,CUR,ISO,SH...
117  '007903107,'AMD,'2007849,US0079031078,ADVANCED...
118  '02079K305,'GOOGL,'BYVY8G0,US02079K3059,ALPHAB...
119  '037833100,'AAPL,'2046251,US0378331005,APPLE I...
120  '038222105,'AMAT,'2046552,US0382221051,APPLIED...
121  '040413205,'ANET,'BL9XPM3,US0404132054,ARISTA ...
122  '11135F101,'AVGO,'BDZ78H9,US11135F1012,BROADCO...
123  '17275R102,'CSCO,'2198163,US17275R1023,CISCO S...
124  '30303M102,'META,'B7TL820,US30303M1027,META PL...
125  '4175